In [1]:
import sys
!{sys.executable} -m pip install altair
!{sys.executable} -m pip install altair_data_server
!{sys.executable} -m pip install prince
!{sys.executable} -m pip install xgboost
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install scikeras

import pandas as pd
import numpy as np
import altair as alt
from xgboost import XGBClassifier
alt.data_transformers.enable('data_server')
import utils

RANDOM_SEED = 42

In [2]:
default_df = pd.read_excel('assets/default of credit card clients.xls', header=1, index_col=0)
default_df.rename(columns={'default payment next month': 'default', 'PAY_0': 'PAY_1'}, inplace=True)
default_df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [3]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split

In [4]:
# data transformation
def_cat_df = default_df.copy()
def_cat_df['AGE_GROUP'] = (def_cat_df.AGE / 10).apply(np.floor).astype(int)
def_cat_df['PAY_1_DUE'] = np.where(def_cat_df['PAY_1'] <= 0, 'YES', 'NO')
def_cat_df['PAY_2_DUE'] = np.where(def_cat_df['PAY_2'] <= 0, 'YES', 'NO')
def_cat_df['PAY_3_DUE'] = np.where(def_cat_df['PAY_3'] <= 0, 'YES', 'NO')
def_cat_df['PAY_4_DUE'] = np.where(def_cat_df['PAY_4'] <= 0, 'YES', 'NO')
def_cat_df['PAY_5_DUE'] = np.where(def_cat_df['PAY_5'] <= 0, 'YES', 'NO')
def_cat_df['PAY_6_DUE'] = np.where(def_cat_df['PAY_6'] <= 0, 'YES', 'NO')
def_cat_df = pd.get_dummies(def_cat_df, columns=['PAY_1_DUE', 'PAY_2_DUE', 'PAY_3_DUE', 'PAY_4_DUE', 'PAY_5_DUE', 'PAY_6_DUE'], drop_first=True)
def_cat_df.drop(columns=['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'AGE'], inplace=True)
def_cat_df['SEX'] = np.where(def_cat_df['SEX'] == 1, 'MALE', 'FEMALE')
def_cat_df = pd.get_dummies(def_cat_df, columns=['SEX'], drop_first=True)
def_cat_df = pd.get_dummies(def_cat_df, columns=['EDUCATION', 'MARRIAGE', 'AGE_GROUP'])
def_cat_df.head()

,LIMIT_BAL,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,...,MARRIAGE_0,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3,AGE_GROUP_2,AGE_GROUP_3,AGE_GROUP_4,AGE_GROUP_5,AGE_GROUP_6,AGE_GROUP_7
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,3913,3102,689,0,0,0,0,689,0,...,0,1,0,0,1,0,0,0,0,0
2,120000,2682,1725,2682,3272,3455,3261,0,1000,1000,...,0,0,1,0,1,0,0,0,0,0
3,90000,29239,14027,13559,14331,14948,15549,1518,1500,1000,...,0,0,1,0,0,1,0,0,0,0
4,50000,46990,48233,49291,28314,28959,29547,2000,2019,1200,...,0,1,0,0,0,1,0,0,0,0
5,50000,8617,5670,35835,20940,19146,19131,2000,36681,10000,...,0,1,0,0,0,0,0,1,0,0


## Random Forest Classifier experiment

In [5]:
X = def_cat_df.drop(columns=['default'])
y = def_cat_df['default']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_SEED)

rf_clf = RandomForestClassifier(random_state=RANDOM_SEED).fit(X_train, y_train)

utils.get_classifier_summary(clsf_type='Random Forest Classifier', clsf=rf_clf, X_test=X_test, y_test=y_test)

alt.Chart(...)

Confusion matrix


,Positive (predicted),Negative (predicted)
Positive (actual),5470,403
Negative (actual),1052,575


Classifier metrics


,Accuracy,Precision,Recall,F1 score,ROC AUC
0,0.81,0.59,0.35,0.44,0.75


alt.HConcatChart(...)

## Gradient Boosting Classifier experiment (all features)

In [6]:
X = def_cat_df.drop(columns=['default'])
y = def_cat_df['default']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_SEED)

gbc = GradientBoostingClassifier(max_depth=8, random_state=RANDOM_SEED).fit(X_train, y_train)

utils.get_classifier_summary(clsf_type='Gradient Boosting Classifier', clsf=gbc, X_test=X_test, y_test=y_test)

alt.Chart(...)

Confusion matrix


,Positive (predicted),Negative (predicted)
Positive (actual),5466,407
Negative (actual),1017,610


Classifier metrics


,Accuracy,Precision,Recall,F1 score,ROC AUC
0,0.81,0.6,0.37,0.46,0.76


alt.HConcatChart(...)

## Gradient Boosting Classifier experiment (high importance features only)

In [7]:
## Let's now remove features with low importance and re-run model training on features with higher importance

low_imp_feat = ['SEX_MALE', 'MARRIAGE_0', 'MARRIAGE_1', 'MARRIAGE_2', 'MARRIAGE_3',
                    'EDUCATION_0', 'EDUCATION_1', 'EDUCATION_2', 'EDUCATION_3', 'EDUCATION_4',
                    'EDUCATION_5', 'EDUCATION_6', 'AGE_GROUP_2', 'AGE_GROUP_3', 'AGE_GROUP_4', 'AGE_GROUP_5', 'AGE_GROUP_6', 'AGE_GROUP_7']

X = def_cat_df.drop(columns=['default'] + low_imp_feat)
y = def_cat_df['default']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_SEED)

gbc_high_imp = GradientBoostingClassifier(max_depth=8, random_state=RANDOM_SEED).fit(X_train, y_train)

utils.get_classifier_summary(clsf_type='Gradient Boosting Classifier (high importance features)', clsf=gbc_high_imp, X_test=X_test, y_test=y_test)

alt.Chart(...)

Confusion matrix


,Positive (predicted),Negative (predicted)
Positive (actual),5471,402
Negative (actual),1024,603


Classifier metrics


,Accuracy,Precision,Recall,F1 score,ROC AUC
0,0.81,0.6,0.37,0.46,0.76


alt.HConcatChart(...)

## XGBoost Classifier experiment

In [8]:
def_cat_df = def_cat_df.astype(int)
X = def_cat_df.drop(columns=['default'])
y = def_cat_df['default']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_SEED)
xgbc = XGBClassifier(objective='binary:logistic', use_label_encoder=False, random_state=RANDOM_SEED, gamma=0.25).fit(X_train, y_train)

utils.get_classifier_summary('XGBoost Classifier', xgbc, X_test=X_test, y_test=y_test)

[11:32:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


alt.Chart(...)

Confusion matrix


,Positive (predicted),Negative (predicted)
Positive (actual),5479,394
Negative (actual),1027,600


Classifier metrics


,Accuracy,Precision,Recall,F1 score,ROC AUC
0,0.81,0.6,0.37,0.46,0.76


alt.HConcatChart(...)

## Neural Network classifier experiment

In [9]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential

numeric_cols = ['LIMIT_BAL', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
                'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
                'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

def_cat_df = def_cat_df.astype(int)
X = def_cat_df.drop(columns=['default'])
y = def_cat_df['default']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_SEED)

scaler = StandardScaler()
# scaler.fit(X)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
scaler.fit(X[numeric_cols])
X_train_num_scaled = scaler.transform(X_train[numeric_cols])
X_train = np.concatenate([X_train_num_scaled, X_train.drop(columns=numeric_cols).to_numpy()], axis=1)
X_test_num_scaled = scaler.transform(X_test[numeric_cols])
X_test = np.concatenate([X_test_num_scaled, X_test.drop(columns=numeric_cols).to_numpy()], axis=1)

def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(0.2, seed=RANDOM_SEED))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=32, verbose=1)
model.fit(X_train, y_train, validation_data=(X_test, y_test))

<ipython-input-9-9d9de0d0b32a>:34: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=32, verbose=1)


Epoch 1/20
704/704 [==============================] - 4s 4ms/step - loss: 0.4785 - accuracy: 0.7936 - val_loss: 0.4520 - val_accuracy: 0.8001
Epoch 2/20
704/704 [==============================] - 3s 5ms/step - loss: 0.4555 - accuracy: 0.8061 - val_loss: 0.4462 - val_accuracy: 0.8009
Epoch 3/20
704/704 [==============================] - 3s 5ms/step - loss: 0.4501 - accuracy: 0.8064 - val_loss: 0.4450 - val_accuracy: 0.8043
Epoch 4/20
704/704 [==============================] - 4s 5ms/step - loss: 0.4505 - accuracy: 0.8065 - val_loss: 0.4437 - val_accuracy: 0.8041
Epoch 5/20
704/704 [==============================] - 4s 5ms/step - loss: 0.4486 - accuracy: 0.8066 - val_loss: 0.4429 - val_accuracy: 0.8028
Epoch 6/20
704/704 [==============================] - 3s 5ms/step - loss: 0.4470 - accuracy: 0.8098 - val_loss: 0.4442 - val_accuracy: 0.8047
Epoch 7/20
704/704 [==============================] - 3s 5ms/step - loss: 0.4450 - accuracy: 0.8088 - val_loss: 0.4404 - val_accuracy: 0.8068
Epoch 